**Босиев Михаил Олегович**

 Название статьи - **Setting up a prediction problem dota 2**
 
 Ссылка: https://www.kaggle.com/devinanzelmo/setting-up-a-prediction-problem-dota-2


**Setting up a prediction problem**

This notebook sets up the problem of predicting the match outcome given the history of each player involved in the match. I go through my thought process as I try to avoid leaks

**Настройка задачи прогнозирования**

Этот блокнот ставит задачу прогнозирования исхода матча с учетом истории каждого игрока, участвовавшего в матче. Я продумываю свой мыслительный процесс, пытаясь избежать утечек

In [2]:
import pandas as pd
import numpy as np
from sklearn import ensemble 
from sklearn import metrics

# это должен быть простой пример, поэтому используются только совпадения и игроки
matches = pd.read_csv('../input/match.csv', index_col=0)
players = pd.read_csv('../input/players.csv')

test_labels = pd.read_csv('../input/test_labels.csv', index_col=0)
test_players = pd.read_csv('../input/test_player.csv')

train_labels = matches['radiant_win'].astype(int)

FileNotFoundError: ignored

**Predicting Match Outcome**

In this problem we are asking the questions: which team will win? It is important to consider when the question is being asked. Most frequently this is asked before the match starts, but it could also be asked after the match has be running for 10 or 15 minutes. It could be asked before hero selection, and all that is known is the identity of the competitors. It could also be asked after hero selection in which case the hero composition of each team would be something to consider. An additional case to consider would be predicting the outcome based only on the heros involved not accounting for the players identities.

The important point is that a time and set of conditions need to be picked before trying to solve the problem. Here we will try to predict the outcome of a match when only the player identities are known, but before hero selection or any gameplay starts.

Any information only available after we ask the question is off limits. This means any details at all about events in the match should be excluded as well as any information about future matches.

**Прогноз исхода матча**

В данной задаче мы задаемся вопросом: какая команда победит?  Чаще всего его спрашивают перед началом матча, но его также можно спросить после того, как матч длится 10 или 15 минут. Его можно спросить перед выбором героя, и все, что известно, - это личности участников. Его также можно было спросить после выбора героя, и в этом случае следует учитывать состав героев каждой команды. Еще один случай, который следует рассмотреть, - это прогнозирование результата, основанное только на задействованных героях, без учета личности игроков.

Важным моментом является то, что необходимо выбрать время и набор условий, прежде чем пытаться решить эту проблему. Здесь мы попытаемся предсказать исход матча, когда известны только личности игроков, но до выбора героя или начала любого игрового процесса.

Любая информация, доступная только после того, как мы зададим вопрос, запрещена. Это означает, что любые подробности о событиях в матче должны быть исключены, а также любая информация о будущих матчах.


In [ ]:
#взглянем на данные матча
matches.head()

Of these variables only game_mode, cluster, and perhaps start_time are possible to determine before the match starts. None of them seem like useful variables if the goal is to use players past performance to predict the match outcome.

Radiant_win is the target variable we are trying to predict. It is pretty easy to see that a time based split is probably best here for validation. By holding out future we reduce the likelyhood of accidently introducing leakage.

Из этих переменных до начала матча можно определить только game_mode, cluster и, возможно, start_time. Ни одна из них не кажется полезной переменной, если цель состоит в том, чтобы использовать прошлые успехи игроков для прогнозирования исхода матча.

Radiant_win - это целевая переменная, которую мы пытаемся предсказать. Довольно легко увидеть, что разделение по времени, вероятно, лучше всего подходит для проверки. 

In [ ]:
# поскольку это простой пример, я буду использовать очень простые функции, которые, вероятно, не очень хороши.
feature_columns = players.iloc[:3,4:17].columns.tolist()
feature_columns

In [ ]:
player_groups = players.groupby('account_id')

# Это просто среднее из приведенных выше значений, по одному для каждой учетной записи.
feature_components = player_groups[feature_columns].mean()

In [ ]:
# account_id 0 включен, даже если он представляет более одной учетной записи
# это средняя статистика для игроков, которые скрывают идентификаторы своих аккаунтов
feгоature_components.head()

In [ ]:
# теперь, чтобы построить функции match_level из компонентов
# account_id необходим для присоединения к feature_components
train_ids = players[['match_id','account_id']]
test_ids = test_players[['match_id','account_id']]

In [ ]:
# добавить данные компонента игрока к полному матчу и данным игрока
# обратите внимание, если игрока нет в наборе поездов, но он появляется в наборе тестов,
# в него будут вставлены значения nan

train_feat_comp = pd.merge(train_ids, feature_components,
                           how='left', left_on='account_id' ,
                           right_index=True)

test_feat_comp = pd.merge(test_ids, feature_components, 
                          how='left', left_on='account_id',
                          right_index=True)

In [ ]:
# когда соединение выполнено,это больше не нужно
train_feat_comp.drop(['account_id'], axis=1, inplace=True)
test_feat_comp.drop(['account_id'], axis=1, inplace=True)

# это в основном сглаживает все совпадение, удаляет избыточные идентификаторы совпадений, 
# а затем удаляет ненужный мультииндекс. 
# Есть ли лучший способ сделать это?
def unstack_simplify(df):
    return df.unstack().iloc[10:].reset_index(drop=True)

In [ ]:
# сгруппировать по матчу, затем объединить данные по всем игрокам в матче в одну строку
test_feat_group = test_feat_comp.groupby('match_id')
test_feats = test_feat_group.apply(unstack_simplify)

In [ ]:
train_feat_group = train_feat_comp.groupby('match_id')
train_feats = train_feat_group.apply(unstack_simplify)

In [ ]:
test_feats.head()

In [ ]:
for i in range(0,40, 10):
    print(test_feats.iloc[0,i:i+10],'\n')

Unstack is interleaving the data of different players the above is to visually check that the nans are showing up in a regular pattern. To make sure I didn't make a mistake.

Below you can see that most matches in the test set have players not in the train set. and this is not accounting for hidding account_ids

Unstack - это чередование данных разных игроков. Вышеупомянутое предназначено для визуальной проверки регулярного отображения NaN-ов. Чтобы убедиться, что я не ошибся.

Ниже вы можете увидеть, что в большинстве матчей в тестовой наборе есть игроки, не входящие в состав Train. И это не учитывает скрытие account_ids

In [ ]:
row_nans = test_feats.isnull().sum(axis=1)
nan_counts = row_nans.value_counts()
nan_counts = nan_counts.reset_index()

nan_counts.columns = ['num_missing_players','count']
nan_counts.loc[:, 'num_missing_players'] =(nan_counts.loc[:,'num_missing_players']/12).astype(int)
nan_counts

# подсчет количества игроков, отсутствующих в матче, потому что их не было в 
# train


In [ ]:
rf = ensemble.RandomForestClassifier(n_estimators=150, n_jobs=-1)
rf.fit(train_feats,train_labels) 


# это плохой способ справиться с отсутствующими значениями 
test_feats.replace(np.nan, 0, inplace=True)

test_probs = rf.predict_proba(test_feats)
test_preds = rf.predict(test_feats)

In [ ]:
metrics.log_loss(test_labels.values.ravel(), test_probs[:,1])

In [ ]:
metrics.roc_auc_score(test_labels.values, test_probs[:,1])

In [ ]:
print(metrics.classification_report(test_labels.values, test_preds))

Having mostly just competed on kaggle, now i have to think about what the metrics mean;) I would say the performance is no where near as good as I would like but with the features I used that is to be expected.

I am more concerned about whether this is the right approach to predicting match outcomes(or i have a bug:)) from player histories. It also seems likely given the number of missing players in the test set that a larger dataset would be useful.

There are other tasks besides predicting match outcomes like predicting win rate, which should be reasonably easy to set up.

В основном я только что соревновался на kaggle, и теперь мне нужно подумать о том, что означают показатели;) Я бы сказал, что производительность далеко не так хороша, как мне хотелось бы, но с функциями, которые я использовал, этого следовало ожидать.

Меня больше беспокоит, является ли это правильным подходом к прогнозированию результатов матча (или у меня есть ошибка :)) на основе историй игроков. Также кажется вероятным, учитывая количество отсутствующих игроков в тестовом наборе, что было бы полезно использовать больший набор данных.

Помимо прогнозирования результатов матча, есть и другие задачи, такие как прогнозирование процента побед, которые должно быть достаточно легко настроить.